In [ ]:
!pip3 install -U selenium
!pip3 install webdriver-manager

!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg --install google-chrome-stable_current_amd64.deb
!apt-get --fix-broken install -y
!apt-get install -y xvfb
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 28.3 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,276 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")


import re, time,csv


In [ ]:
"""
Extracts all the story links from the current page and writes to the specified writer
"""
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, StaleElementReferenceException
)

def get_info_from_page(writer: csv.writer):
    # get all story list items on the current page
    results = driver.find_elements(By.CSS_SELECTOR, 'li[data-uri*="/_components/card/instances/search-"]')
    print('Result Number', len(results))

    def get_text(el):
        # prefer visible text; fall back to innerText/textContent
        return (el.text or el.get_attribute("innerText") or el.get_attribute("textContent") or "").strip()

    for idx, result in enumerate(results):
        headline, link, date = 'NA', 'NA', 'NA'

        try:
            # ensure the result itself is attached/visible
            try:
                WebDriverWait(driver, 5).until(lambda d: result.is_displayed())
            except StaleElementReferenceException:
                # if the result went stale, re-fetch it by index
                results_refetched = driver.find_elements(By.CSS_SELECTOR, 'li[data-uri*="/_components/card/instances/search-"]')
                if idx < len(results_refetched):
                    result = results_refetched[idx]

            # Extract the link - look for the anchor tag
            try:
                link_element = result.find_element(By.CSS_SELECTOR, 'a.container__link')
                link = link_element.get_attribute('href')
                print(f"Link: {link}")
            except (NoSuchElementException, StaleElementReferenceException) as e:
                print(f'Could not extract link: {type(e).__name__}')

            # Extract the date - look for the div with container__date class
            try:
                date_element = result.find_element(By.CSS_SELECTOR, 'div.container__date')
                date = get_text(date_element)
                print(f"Date: {date}")
            except (NoSuchElementException, StaleElementReferenceException) as e:
                print(f'Could not extract date: {type(e).__name__}')

            # find headline element inside this result
            headline_box = result.find_element(By.CSS_SELECTOR, 'span.container__headline-text')

            # wait until text is non-empty (with fallback)
            WebDriverWait(driver, 10).until(lambda d: get_text(headline_box) != "")

            # scrolling sometimes flips visibility state in virtualized lists
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", headline_box)

            headline = get_text(headline_box)

            print(f"Headline: {headline}")
            print("-" * 80)

        except (TimeoutException, NoSuchElementException, StaleElementReferenceException) as e:
            print('could not extract headline:', type(e).__name__)

        # write a new row for this story
        writer.writerow([link, date, headline])

In [ ]:

def scrape_query(query:str,
                delay:int = 5 # number of seconds to wait
                ):

    #create a new csv writer for the story links
    fw=open(query+'_links.csv','w')
    writer=csv.writer(fw,lineterminator='\n')
    writer.writerow(['link','date','headline'])

    url='https://www.cnn.com/search?size=10&q='+query # create the url

    driver.get(url) # visit the page
    time.sleep(2)
    page_cnt=1 # keep track of page count

    while True: # keep going until there are no more pages

        print('page',page_cnt) # print current page count

        page_cnt+=1 # increment

        #extract and write the links from the current page
        get_info_from_page(writer)

        #wait until the next button appears
        next_button = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[class*="pagination-arrow pagination-arrow-right"]')))

        #check if the next button is clickable
        if 'text-active' not in next_button.get_attribute('class'):
            print('no more pages')
            break

        #click the next button and wait
        #next_button.click()
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(delay)

    fw.close()

In [ ]:

# Set up ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


query='bitcoin%20bubble' # specify a query


scrape_query(query)

page 1
Result Number 10
The AI bubble may not be bursting, but tariff chaos is sure helping to deflate it
The origins of bubble tea, one of Taiwan’s most beloved beverages
Is the tea bubble bursting? ChaPanda shares plunge in Hong Kong IPO
This investor predicted the dot-com bust. He thinks AI is a bubble that will ‘deflate’
Two of China’s bubble tea giants are gearing up for IPOs
'What's happening?': Woman's sudsy toilet goes viral
Financial Times Associate Editor: We're in middle of an AI bubble
'Happy accident': Bear delights in bubble bath on zoo's cleaning day
‘The Beanie Bubble’ nostalgically looks back at the ‘Babies’ boom that went bust
This investor can spot market bubbles. Now he’s warning of an ‘ominous’ bust


KeyboardInterrupt: 